<a href="https://colab.research.google.com/github/marianatiele/etl_aws_combustiveis/blob/main/Notebook_GOLD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Camada Gold**

Os dados refinados da camada Silver, armazenados no S3, serão modelados e carregados no banco de dados RDS (PostgreSQL). Essa estruturação permitirá análises avançadas no Power BI Desktop.

## **Instalando o Pyspark**

In [1]:
!pip install findspark

In [2]:
!pip install pyngrok

In [3]:
# instalar as dependências

!apt-get update -qq
!apt-get install openjdk-11-jdk-headless -qq > /dev/null  # Instalação do Java

# Baixar a versão do Spark desejada (Spark 3.5.4 com Hadoop 3)
!wget -q https://archive.apache.org/dist/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz

# Extrair os arquivos do Spark
!tar xf spark-3.5.4-bin-hadoop3.tgz


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [4]:
!java -version

openjdk version "11.0.28" 2025-07-15
OpenJDK Runtime Environment (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)


In [5]:
# # Drive jdbc postgreSQL
!wget https://jdbc.postgresql.org/download/postgresql-42.2.27.jar



--2025-08-02 03:04:25--  https://jdbc.postgresql.org/download/postgresql-42.2.27.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1006852 (983K) [application/java-archive]
Saving to: ‘postgresql-42.2.27.jar’

postgresql-42.2.27. 100%[===================>] 983.25K  1.20MB/s    in 0.8s    

2025-08-02 03:04:27 (1.20 MB/s) - ‘postgresql-42.2.27.jar’ saved [1006852/1006852]



In [6]:
!ls -lh postgresql-42.2.27.jar


-rw-r--r-- 1 root root 984K Nov 23  2022 postgresql-42.2.27.jar


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyngrok import ngrok
import findspark
import os

In [8]:
# configurar as variáveis de ambiente
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"
sys.path.append("/content/spark-3.5.4-bin-hadoop3/python")
sys.path.append("/content/spark-3.5.4-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip")
# RDS PostgreSQL
os.environ["PYSPARK_SUBMIT_ARGS"] = "--jars postgresql-42.2.27.jar pyspark-shell"

# S3
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

aws_access_key_id = "SUA KEY AWS"
aws_secret_access_key = "SUA SECRET KEY AWS"
region_name = "us-east-1"

In [9]:
findspark.init()

## **Iniciando Sessão Spark e Conexão com a S3**

In [10]:
spark = SparkSession.builder \
    .appName("Spark com S3 ") \
    .config("spark.jars", "/content/postgresql-42.2.27.jar")\
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1")\
    .config("spark.hadoop.fs.s3a.access.key", aws_access_key_id) \
    .config("spark.hadoop.fs.s3a.secret.key", aws_secret_access_key) \
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()


In [11]:
dados = spark.read.parquet('s3a://datalake-combustiveis-br/Silver/preco-gasolina-etanol-01', header=True)

In [42]:
dados.show(2)

+------+------+---------+------+--------------------+---------+------+-----------+------------------+-----------+--------+--------------------+
|REGIAO|ESTADO|MUNICIPIO|BAIRRO|                 RUA|      CEP|NUMERO|DATA_COLETA|           PRODUTO|VALOR_VENDA|BANDEIRA|             REVENDA|
+------+------+---------+------+--------------------+---------+------+-----------+------------------+-----------+--------+--------------------+
|    NE|    CE|   SOBRAL|CENTRO|RUA TABELIÃO IDEL...|62010-000|   455| 01/01/2025|          GASOLINA|       6.29|  RAIZEN|ECONOGÁS DO BRASI...|
|    NE|    CE|   SOBRAL|CENTRO|RUA TABELIÃO IDEL...|62010-000|   455| 01/01/2025|GASOLINA ADITIVADA|       6.49|  RAIZEN|ECONOGÁS DO BRASI...|
+------+------+---------+------+--------------------+---------+------+-----------+------------------+-----------+--------+--------------------+
only showing top 2 rows



In [13]:
dados_2 = spark.read.parquet('s3a://datalake-combustiveis-br/Silver/preco-gasolina-etanol-02', header=True)
dados_3 = spark.read.parquet('s3a://datalake-combustiveis-br/Silver/preco-gasolina-etanol-03', header=True)

In [43]:
dados_2.show(2)

+------+------+---------------+------+--------------------+---------+------+-----------+------------------+-----------+--------+--------------------+
|REGIAO|ESTADO|      MUNICIPIO|BAIRRO|                 RUA|      CEP|NUMERO|DATA_COLETA|           PRODUTO|VALOR_VENDA|BANDEIRA|             REVENDA|
+------+------+---------------+------+--------------------+---------+------+-----------+------------------+-----------+--------+--------------------+
|     N|    AC|CRUZEIRO DO SUL|CENTRO|AVENIDA BOULEVARD...|69980-000|  1031| 03/02/2025|          GASOLINA|       7.27|   VIBRA|CARDINAL E OLIVEI...|
|     N|    AC|CRUZEIRO DO SUL|CENTRO|AVENIDA BOULEVARD...|69980-000|  1031| 03/02/2025|GASOLINA ADITIVADA|       7.73|   VIBRA|CARDINAL E OLIVEI...|
+------+------+---------------+------+--------------------+---------+------+-----------+------------------+-----------+--------+--------------------+
only showing top 2 rows



In [44]:
dados_3.show(2)

+------+------+---------+--------------+--------------------+---------+------+-----------+--------+-----------+--------+--------------------+
|REGIAO|ESTADO|MUNICIPIO|        BAIRRO|                 RUA|      CEP|NUMERO|DATA_COLETA| PRODUTO|VALOR_VENDA|BANDEIRA|             REVENDA|
+------+------+---------+--------------+--------------------+---------+------+-----------+--------+-----------+--------+--------------------+
|    SE|    SP| SOROCABA|JARDIM ZULMIRA|RUA HUMBERTO DE C...|18061-000|   306| 03/03/2025|GASOLINA|       5.65|  BRANCA|COMPETRO COMERCIO...|
|    SE|    SP| SOROCABA|JARDIM ZULMIRA|RUA HUMBERTO DE C...|18061-000|   306| 03/03/2025|  ETANOL|       3.78|  BRANCA|COMPETRO COMERCIO...|
+------+------+---------+--------------+--------------------+---------+------+-----------+--------+-----------+--------+--------------------+
only showing top 2 rows



## **Análises**

In [28]:
# Regiao Nordeste com maiores preço de gasolina
(dados.select('REGIAO', 'ESTADO', 'MUNICIPIO', 'VALOR_VENDA')
    .filter((dados.REGIAO== 'NE') & (dados.VALOR_VENDA >= 6))
    .show()
)

+------+------+---------+-----------+
|REGIAO|ESTADO|MUNICIPIO|VALOR_VENDA|
+------+------+---------+-----------+
|    NE|    CE|   SOBRAL|       6.29|
|    NE|    CE|   SOBRAL|       6.49|
|    NE|    CE|   SOBRAL|       6.53|
|    NE|    CE|   SOBRAL|       6.83|
|    NE|    CE|   SOBRAL|       6.69|
|    NE|    CE|   SOBRAL|       6.79|
|    NE|    CE|   SOBRAL|       6.69|
|    NE|    CE|   SOBRAL|       6.69|
|    NE|    CE|   SOBRAL|       6.69|
|    NE|    CE|   SOBRAL|       6.99|
|    NE|    CE|   SOBRAL|       6.64|
|    NE|    CE|   SOBRAL|       6.84|
|    NE|    RN|    CAICO|       6.19|
|    NE|    RN|    CAICO|       6.41|
|    NE|    RN|    CAICO|       6.37|
|    NE|    RN|    CAICO|       6.45|
|    NE|    CE|   SOBRAL|       6.64|
|    NE|    CE|   SOBRAL|       6.64|
|    NE|    CE|   SOBRAL|       6.38|
|    NE|    CE|   SOBRAL|       6.68|
+------+------+---------+-----------+
only showing top 20 rows



In [29]:
# Regiao Sudeste com maiores preço de gasolina
(dados.select('REGIAO', 'ESTADO', 'MUNICIPIO', 'VALOR_VENDA')
    .filter((dados.REGIAO== 'SE') & (dados.VALOR_VENDA >= 6))
    .show()
)

+------+------+--------------------+-----------+
|REGIAO|ESTADO|           MUNICIPIO|VALOR_VENDA|
+------+------+--------------------+-----------+
|    SE|    MG|          UBERLANDIA|       6.05|
|    SE|    MG|          UBERLANDIA|       6.79|
|    SE|    MG|          UBERLANDIA|       6.09|
|    SE|    MG|          UBERLANDIA|       6.37|
|    SE|    RJ|CAMPOS DOS GOYTAC...|       6.49|
|    SE|    RJ|CAMPOS DOS GOYTAC...|       6.69|
|    SE|    RJ|CAMPOS DOS GOYTAC...|       6.42|
|    SE|    RJ|CAMPOS DOS GOYTAC...|       6.49|
|    SE|    RJ|CAMPOS DOS GOYTAC...|       6.62|
|    SE|    RJ|CAMPOS DOS GOYTAC...|       6.49|
|    SE|    RJ|CAMPOS DOS GOYTAC...|       6.56|
|    SE|    RJ|CAMPOS DOS GOYTAC...|       6.43|
|    SE|    RJ|CAMPOS DOS GOYTAC...|       6.53|
|    SE|    RJ|CAMPOS DOS GOYTAC...|       6.49|
|    SE|    RJ|CAMPOS DOS GOYTAC...|       6.56|
|    SE|    RJ|CAMPOS DOS GOYTAC...|       6.42|
|    SE|    RJ|CAMPOS DOS GOYTAC...|       6.49|
|    SE|    RJ|CAMPO

In [30]:
# Regiao Norte com maiores preço de gasolina
(dados.select('REGIAO', 'ESTADO', 'MUNICIPIO', 'VALOR_VENDA')
    .filter((dados.REGIAO== 'N') & (dados.VALOR_VENDA >= 6))
    .show()
)

+------+------+--------------------+-----------+
|REGIAO|ESTADO|           MUNICIPIO|VALOR_VENDA|
+------+------+--------------------+-----------+
|     N|    AC|          RIO BRANCO|       7.39|
|     N|    AC|          RIO BRANCO|       7.45|
|     N|    AC|          RIO BRANCO|       7.49|
|     N|    AC|          RIO BRANCO|       7.49|
|     N|    AC|          RIO BRANCO|       7.45|
|     N|    AC|          RIO BRANCO|       7.49|
|     N|    AC|          RIO BRANCO|       7.49|
|     N|    AC|          RIO BRANCO|       7.48|
|     N|    AC|          RIO BRANCO|       7.63|
|     N|    AC|          RIO BRANCO|       7.44|
|     N|    AC|          RIO BRANCO|       7.44|
|     N|    AC|          RIO BRANCO|       7.45|
|     N|    AC|          RIO BRANCO|       7.49|
|     N|    TO|PARAISO DO TOCANTINS|       6.44|
|     N|    TO|PARAISO DO TOCANTINS|       6.59|
|     N|    TO|PARAISO DO TOCANTINS|       6.25|
|     N|    TO|PARAISO DO TOCANTINS|       6.35|
|     N|    TO|PARAI

In [31]:
# Regiao Centro Oeste  com maiores preço de gasolina
(dados.select('REGIAO', 'ESTADO', 'MUNICIPIO', 'VALOR_VENDA')
    .filter((dados.REGIAO== 'CO') & (dados.VALOR_VENDA >= 6))
    .show()
)

+------+------+----------+-----------+
|REGIAO|ESTADO| MUNICIPIO|VALOR_VENDA|
+------+------+----------+-----------+
|    CO|    DF|  BRASILIA|       6.13|
|    CO|    DF|  BRASILIA|       6.73|
|    CO|    GO|  ANAPOLIS|       6.19|
|    CO|    GO| RIO VERDE|       6.29|
|    CO|    GO| RIO VERDE|       6.04|
|    CO|    GO| RIO VERDE|       6.09|
|    CO|    MS|PONTA PORA|       6.56|
|    CO|    MS|PONTA PORA|       6.67|
|    CO|    MS|PONTA PORA|       6.19|
|    CO|    MS|PONTA PORA|       6.19|
|    CO|    MS|PONTA PORA|       6.19|
|    CO|    MS|PONTA PORA|       6.29|
|    CO|    MS|PONTA PORA|       6.34|
|    CO|    MT|    CUIABA|       6.29|
|    CO|    MT|    CUIABA|       6.39|
|    CO|    MT|    CUIABA|       6.17|
|    CO|    MT|    CUIABA|       6.17|
|    CO|    GO|  LUZIANIA|       6.19|
|    CO|    GO| RIO VERDE|       6.29|
|    CO|    MT|    CUIABA|       6.29|
+------+------+----------+-----------+
only showing top 20 rows



In [32]:
# Regiao SUL com maiores preço de gasolina
(dados.select('REGIAO', 'ESTADO', 'MUNICIPIO', 'VALOR_VENDA')
    .filter((dados.REGIAO== 'S') & (dados.VALOR_VENDA >= 6))
    .show()
)

+------+------+---------+-----------+
|REGIAO|ESTADO|MUNICIPIO|VALOR_VENDA|
+------+------+---------+-----------+
|     S|    RS|  VACARIA|       6.39|
|     S|    RS|  VACARIA|       6.52|
|     S|    PR|APUCARANA|       6.27|
|     S|    PR|APUCARANA|       6.42|
|     S|    PR|APUCARANA|       6.29|
|     S|    PR|APUCARANA|       6.27|
|     S|    PR| LONDRINA|       6.29|
|     S|    PR| LONDRINA|       6.49|
|     S|    PR| LONDRINA|       6.19|
|     S|    PR| LONDRINA|       6.49|
|     S|    RS|   GUAIBA|       6.19|
|     S|    PR|APUCARANA|       6.29|
|     S|    PR|APUCARANA|       6.39|
|     S|    PR|APUCARANA|       6.19|
|     S|    PR|APUCARANA|       6.22|
|     S|    PR|APUCARANA|       6.27|
|     S|    PR|APUCARANA|       6.44|
|     S|    RS|  VACARIA|       6.19|
|     S|    RS|  VACARIA|       6.59|
|     S|    PR|ARAPONGAS|       6.19|
+------+------+---------+-----------+
only showing top 20 rows



In [33]:
(dados.select('REGIAO','VALOR_VENDA',  'DATA_COLETA')
    .groupBy('REGIAO', 'DATA_COLETA')
    .agg({'VALOR_VENDA':'mean'})
    .show()
)

+------+-----------+------------------+
|REGIAO|DATA_COLETA|  avg(VALOR_VENDA)|
+------+-----------+------------------+
|    NE| 24/01/2025| 5.782191768084487|
|    CO| 24/01/2025| 5.348157910924209|
|     S| 13/01/2025| 5.844280119226755|
|    NE| 04/01/2025| 5.851232822627237|
|    CO| 16/01/2025| 5.541209657346049|
|     S| 23/01/2025|5.8438698241155445|
|     S| 24/01/2025| 5.772599949836731|
|    CO| 06/01/2025| 5.503999987284343|
|     S| 14/01/2025|5.8402999711036685|
|    CO| 22/01/2025| 5.393936949452077|
|    CO| 03/01/2025| 5.588111090660095|
|    CO| 10/01/2025| 5.291249970595042|
|     N| 16/01/2025| 6.581739011018173|
|    NE| 02/01/2025| 5.857819871314894|
|    NE| 09/01/2025| 5.713702676747296|
|    CO| 09/01/2025| 5.366333287954331|
|    SE| 06/01/2025|5.4552806575677675|
|    SE| 23/01/2025| 5.460543181572431|
|    NE| 10/01/2025|5.7353029865207095|
|    NE| 03/01/2025| 5.849838682720738|
+------+-----------+------------------+
only showing top 20 rows



In [34]:
(dados_2.select('REGIAO','VALOR_VENDA',  'DATA_COLETA')
    .groupBy('REGIAO', 'DATA_COLETA')
    .agg({'VALOR_VENDA':'mean'})
    .show()
)

+------+-----------+------------------+
|REGIAO|DATA_COLETA|  avg(VALOR_VENDA)|
+------+-----------+------------------+
|     S| 21/02/2025| 6.048425152545839|
|    CO| 21/02/2025| 5.730000019073486|
|    NE| 11/02/2025| 5.969538714612982|
|     S| 20/02/2025| 5.993476982774405|
|    NE| 04/02/2025| 5.956163838367001|
|    CO| 17/02/2025| 5.863464901321812|
|    NE| 06/02/2025| 6.067214666067747|
|    SE| 10/02/2025| 5.653049493850986|
|    CO| 14/02/2025| 5.143333315849304|
|    NE| 24/02/2025| 6.003356450961696|
|    SE| 07/02/2025|5.3796581203102045|
|    SE| 19/02/2025| 5.572693093584574|
|    SE| 21/02/2025| 5.372597338317276|
|    CO| 20/02/2025|  5.54515142513044|
|    SE| 24/02/2025| 5.645324213942272|
|     N| 24/02/2025|  6.36373412910896|
|     N| 05/02/2025| 6.651951169579979|
|    CO| 04/02/2025| 5.821314104856589|
|    SE| 13/02/2025| 5.576805232129421|
|    SE| 18/02/2025| 5.672410902743905|
+------+-----------+------------------+
only showing top 20 rows



In [35]:
(dados_3.select('REGIAO','VALOR_VENDA',  'DATA_COLETA')
    .groupBy('REGIAO', 'DATA_COLETA')
    .agg({'VALOR_VENDA':'mean'})
    .show()
)

+------+-----------+------------------+
|REGIAO|DATA_COLETA|  avg(VALOR_VENDA)|
+------+-----------+------------------+
|    CO| 21/03/2025|  5.18999981880188|
|    CO| 07/03/2025| 5.508999933515276|
|    SE| 17/03/2025| 5.628216752361307|
|     N| 04/03/2025| 6.496363567583488|
|     N| 12/03/2025| 6.745479413907822|
|    CO| 04/03/2025| 5.984444429844986|
|     S| 11/03/2025|6.0203092348944285|
|    SE| 24/03/2025| 5.613237829462996|
|     N| 24/03/2025|  6.34621990013778|
|     N| 06/03/2025|6.7142381055014475|
|    SE| 07/03/2025| 5.643927365639816|
|     N| 10/03/2025| 6.360441600486683|
|     S| 07/03/2025|  6.00428566556846|
|     S| 24/03/2025| 6.044023770661581|
|    NE| 20/03/2025| 6.036182757346861|
|     N| 07/03/2025| 6.621481459817769|
|    SE| 06/03/2025| 5.629785070580331|
|     S| 20/03/2025|  6.11878539193497|
|    CO| 03/03/2025| 5.884439818591993|
|    CO| 10/03/2025| 5.786724556764361|
+------+-----------+------------------+
only showing top 20 rows



In [36]:
(dados.select('ESTADO','PRODUTO','VALOR_VENDA',  'REVENDA')
    .groupBy('ESTADO','PRODUTO', 'REVENDA')
    .agg({'VALOR_VENDA':'min'})
    .show()
)

+------+------------------+--------------------+----------------+
|ESTADO|           PRODUTO|             REVENDA|min(VALOR_VENDA)|
+------+------------------+--------------------+----------------+
|    SP|            ETANOL|POSTO DE SERVICOS...|            3.99|
|    RJ|          GASOLINA|VICMAR COMERCIO D...|            6.44|
|    TO|          GASOLINA|MEDEIROS COMERCIO...|             6.2|
|    RJ|GASOLINA ADITIVADA|POSTO JOSE BAPTIS...|            6.49|
|    AL|            ETANOL|REYAUTO COMERCIO ...|            4.49|
|    SP|          GASOLINA|JULIO FERNANDES &...|            5.89|
|    SP|GASOLINA ADITIVADA|     LIMA E DIB LTDA|            6.69|
|    SP|          GASOLINA|AUTO POSTO TREVO ...|            6.49|
|    SP|          GASOLINA|CENTRO AUTOMOTIVO...|            5.97|
|    SC|GASOLINA ADITIVADA|AUTO POSTO R&R CE...|             6.5|
|    SC|            ETANOL|POSTO SONHO MEU LTDA|            4.39|
|    SP|            ETANOL|AUTO POSTO GUANAB...|            3.79|
|    SP|  

In [37]:
(dados.select('ESTADO','PRODUTO','VALOR_VENDA',  'REVENDA')
    .groupBy('ESTADO','PRODUTO', 'REVENDA')
    .agg({'VALOR_VENDA':'max'})
    .show()
)

+------+------------------+--------------------+----------------+
|ESTADO|           PRODUTO|             REVENDA|max(VALOR_VENDA)|
+------+------------------+--------------------+----------------+
|    SP|            ETANOL|POSTO DE SERVICOS...|            4.09|
|    RJ|          GASOLINA|VICMAR COMERCIO D...|            6.54|
|    TO|          GASOLINA|MEDEIROS COMERCIO...|             6.2|
|    RJ|GASOLINA ADITIVADA|POSTO JOSE BAPTIS...|            6.59|
|    AL|            ETANOL|REYAUTO COMERCIO ...|            4.89|
|    SP|          GASOLINA|JULIO FERNANDES &...|            5.89|
|    SP|GASOLINA ADITIVADA|     LIMA E DIB LTDA|            6.69|
|    SP|          GASOLINA|AUTO POSTO TREVO ...|            6.49|
|    SP|          GASOLINA|CENTRO AUTOMOTIVO...|            7.79|
|    SC|GASOLINA ADITIVADA|AUTO POSTO R&R CE...|             6.5|
|    SC|            ETANOL|POSTO SONHO MEU LTDA|            4.39|
|    SP|            ETANOL|AUTO POSTO GUANAB...|            3.99|
|    SP|  

In [38]:
(dados.select('ESTADO','PRODUTO','VALOR_VENDA',  'REVENDA')
    .groupBy('ESTADO','PRODUTO', 'REVENDA')
    .agg({'VALOR_VENDA':'mean'})
    .show()
)

+------+------------------+--------------------+------------------+
|ESTADO|           PRODUTO|             REVENDA|  avg(VALOR_VENDA)|
+------+------------------+--------------------+------------------+
|    SP|            ETANOL|POSTO DE SERVICOS...|4.0100000381469725|
|    RJ|          GASOLINA|VICMAR COMERCIO D...| 6.509999942779541|
|    TO|          GASOLINA|MEDEIROS COMERCIO...| 6.199999809265137|
|    RJ|GASOLINA ADITIVADA|POSTO JOSE BAPTIS...| 6.515999889373779|
|    AL|            ETANOL|REYAUTO COMERCIO ...| 4.689999938011169|
|    SP|          GASOLINA|JULIO FERNANDES &...| 5.889999866485596|
|    SP|GASOLINA ADITIVADA|     LIMA E DIB LTDA| 6.690000057220459|
|    SP|          GASOLINA|AUTO POSTO TREVO ...| 6.489999771118164|
|    SP|          GASOLINA|CENTRO AUTOMOTIVO...| 6.879999876022339|
|    SC|GASOLINA ADITIVADA|AUTO POSTO R&R CE...|               6.5|
|    SC|            ETANOL|POSTO SONHO MEU LTDA| 4.389999866485596|
|    SP|            ETANOL|AUTO POSTO GUANAB...|

In [39]:
(dados.select('PRODUTO','VALOR_VENDA')
     .groupBy('PRODUTO')
     .avg('VALOR_VENDA')
    .show()
)

+------------------+-----------------+
|           PRODUTO| avg(VALOR_VENDA)|
+------------------+-----------------+
|GASOLINA ADITIVADA|6.379578455277653|
|            ETANOL|4.368401601892578|
|          GASOLINA|6.183742139215518|
+------------------+-----------------+



In [40]:
(dados.select('BANDEIRA', 'PRODUTO')).distinct().show()

+--------------------+------------------+
|            BANDEIRA|           PRODUTO|
+--------------------+------------------+
|          PETROBAHIA|GASOLINA ADITIVADA|
|             CHARRUA|          GASOLINA|
|              TAURUS|          GASOLINA|
|             DIBRAPE|GASOLINA ADITIVADA|
|         PETROBRASIL|GASOLINA ADITIVADA|
|         PETROBRASIL|            ETANOL|
|                  SP|          GASOLINA|
|              TAURUS|GASOLINA ADITIVADA|
| SETTA DISTRIBUIDORA|          GASOLINA|
|         RAIZEN MIME|GASOLINA ADITIVADA|
|             EQUADOR|          GASOLINA|
|              D`MAIS|GASOLINA ADITIVADA|
|                 FAN|          GASOLINA|
|PETROX DISTRIBUIDORA|          GASOLINA|
|         RAIZEN MIME|          GASOLINA|
| MASUT DISTRIBUIDORA|            ETANOL|
|           ROYAL FIC|            ETANOL|
|                 ALE|          GASOLINA|
|              TEMAPE|          GASOLINA|
|              D`MAIS|            ETANOL|
+--------------------+------------

In [41]:
dados.select('REGIAO').distinct().show()

+------+
|REGIAO|
+------+
|    NE|
|     N|
|     S|
|    SE|
|    CO|
+------+



## **Criando as tabelas para o banco de dados**

In [14]:

dados = dados.select('REGIAO','ESTADO', 'MUNICIPIO','BAIRRO', 'RUA', 'CEP','NUMERO', 'DATA_COLETA', 'PRODUTO', 'VALOR_VENDA', 'BANDEIRA', 'REVENDA')

dados_2 = dados_2.select('REGIAO','ESTADO', 'MUNICIPIO','BAIRRO', 'RUA', 'CEP','NUMERO', 'DATA_COLETA', 'PRODUTO', 'VALOR_VENDA', 'BANDEIRA', 'REVENDA')

dados_3 = dados_3.select('REGIAO','ESTADO', 'MUNICIPIO','BAIRRO', 'RUA', 'CEP','NUMERO', 'DATA_COLETA', 'PRODUTO', 'VALOR_VENDA', 'BANDEIRA', 'REVENDA')

In [15]:
columns = dados.columns

dados = dados.select(columns)
ddados_2 = dados_2.select(columns)
dados_3 = dados_3.select(columns)

dados_final = dados.union(dados_2).union(dados_3)

In [47]:
len(dados.columns), len(dados_final.columns), dados_final.count()

(12, 12, 142165)

In [45]:
dados_final.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in dados_final.columns
]).show()

+------+------+---------+------+---+---+------+-----------+-------+-----------+--------+-------+
|REGIAO|ESTADO|MUNICIPIO|BAIRRO|RUA|CEP|NUMERO|DATA_COLETA|PRODUTO|VALOR_VENDA|BANDEIRA|REVENDA|
+------+------+---------+------+---+---+------+-----------+-------+-----------+--------+-------+
|     0|     0|        0|     0|  0|  0|     0|          0|      0|          0|       0|      0|
+------+------+---------+------+---+---+------+-----------+-------+-----------+--------+-------+



In [48]:
len(dados_final.columns), len(dados.columns), len(dados_2.columns), len(dados_3.columns)

(12, 12, 12, 12)

In [16]:
!mkdir dados_csv

In [18]:
dados_final = dados_final.coalesce(1)

dados_final.write.format('csv')\
.mode('overwrite').option('header', 'true')\
.option('inforSchema', 'true').save('/content/dados_csv/tb_dados')

In [19]:
dados_csv = spark.read.format('csv').option('header','true').option('inforSchema', 'true').load('/content/dados_csv/tb_dados')

In [49]:
dados_csv.show(2)

+------+------+---------+------+--------------------+---------+------+-----------+------------------+-----------+--------+--------------------+
|REGIAO|ESTADO|MUNICIPIO|BAIRRO|                 RUA|      CEP|NUMERO|DATA_COLETA|           PRODUTO|VALOR_VENDA|BANDEIRA|             REVENDA|
+------+------+---------+------+--------------------+---------+------+-----------+------------------+-----------+--------+--------------------+
|    NE|    CE|   SOBRAL|CENTRO|RUA TABELIÃO IDEL...|62010-000|   455| 01/01/2025|          GASOLINA|       6.29|  RAIZEN|ECONOGÁS DO BRASI...|
|    NE|    CE|   SOBRAL|CENTRO|RUA TABELIÃO IDEL...|62010-000|   455| 01/01/2025|GASOLINA ADITIVADA|       6.49|  RAIZEN|ECONOGÁS DO BRASI...|
+------+------+---------+------+--------------------+---------+------+-----------+------------------+-----------+--------+--------------------+
only showing top 2 rows



In [50]:
dados_csv.count()

142165

## **Enviando os dados para RDS PostrgreSQL**

In [27]:
dados_csv.write \
    .format("jdbc") \
    .option("url", "ENDPOINT") \
    .option("dbtable", "TABELA") \
    .option("user", "USUÁRIO") \
    .option("password", "SUA SENHA") \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()